In [ ]:

import pymongo
client = pymongo.MongoClient("mongodb://<username>:<password>@ac-fpwcfpu-shard-00-00.dbyavcr.mongodb.net:27017,ac-fpwcfpu-shard-00-01.dbyavcr.mongodb.net:27017,ac-fpwcfpu-shard-00-02.dbyavcr.mongodb.net:27017/?ssl=true&replicaSet=atlas-hm9440-shard-0&authSource=admin&retryWrites=true&w=majority")
db = client.predictive_maintenance

Sensors_data = db.Sensors_data

In [ ]:
from kafka import KafkaConsumer
from pymongo import MongoClient
from json import loads
import pandas as pd
import asyncio
async def receive_data() :
    consumer = KafkaConsumer(
    'machine_details',
     bootstrap_servers=['localhost:9092'],
     auto_offset_reset='earliest',
     enable_auto_commit=True,
    # group_id='IotData',
     value_deserializer=lambda x: loads(x.decode('utf-8')))


    df = pd.DataFrame()
    for message in consumer:
        data =await message.value
        df_iter =await pd.json_normalize(data)
        df =await df.append(df_iter)
        ##send received data from consumer(sensors) to mongodb atlas predictive_maintenance database
        await Sensors_data.insert_one(message.value)